# Lab 002 · Random Forest (Rosario)

In [ ]:
# Preparación del entorno (solo si hace falta)
try:
    import ee
    import geemap.foliumap as geemap
except Exception:
    %pip -q install earthengine-api geemap
    import ee
    import geemap.foliumap as geemap


In [ ]:
# Autenticación e inicialización de Earth Engine
try:
    ee.Initialize(project="ee-cdgidera")
    print("Earth Engine inicializado.")
except Exception:
    print("Autenticación requerida: ejecutá esta celda, seguí el enlace, pegá el código y ejecutá de nuevo.")
    ee.Authenticate()
    ee.Initialize(project="ee-cdgidera")


In [ ]:
import geemap, json, ee

In [ ]:
# Cargar los assets exportados
roi_fc  = ee.FeatureCollection('users/cdg-idera/roi_polygon_2024')
roi     = roi_fc.geometry()

gcps = ee.FeatureCollection('users/cdg-idera/gcps_landcover_2024')

print('ROI features:', roi_fc.size().getInfo())
print('GCPS features:', gcps.size().getInfo())


ROI features: 1
GCPS features: 261


In [14]:
# Paso 1. Sentinel-2 -> filtro -> composite (mediana) -> mostrar composite
# ImageCollection S2
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# Filtros y selección de bandas B.*
filtered = (s2
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
    .filterDate('2024-01-01', '2025-01-01')
    .filterBounds(roi)
    .select('B.*')
)

# Composite mediana y recorte al ROI
composite = filtered.median().clip(roi)

# Visualizar composite ANTES de dividir train/valid
Map = geemap.Map()
Map.centerObject(roi, 10)
rgbVisParams = {'min': 0, 'max': 3000, 'bands': ['B4','B3','B2']}
Map.addLayer(composite, rgbVisParams, 'ROI: región de interés (Composite S2 2024)')
Map

Map(center=[-32.97411051812981, -60.747791764102], controls=(WidgetControl(options=['position', 'transparent_b…

In [17]:
# paso 2) Tranining and validation (60/40) con columna aleatoria
# Asignar número aleatorio en [0,1)
gcp = gcps.randomColumn()  # opcional: .randomColumn('random', seed=0)

trainingGCP   = gcp.filter(ee.Filter.lt('random', 0.6))
validationGCP = gcp.filter(ee.Filter.gte('random', 0.6))

print('Training GCP:', trainingGCP.size().getInfo())
print('Validation GCP:', validationGCP.size().getInfo())

Training GCP: 159
Validation GCP: 102


In [18]:
# Paso 3) extraer pixeles (sampleRegions) para entrenamiento

training = composite.sampleRegions(
    collection=trainingGCP,
    properties=['landcover'],
    scale=10,
    tileScale=16
)

print('Training samples (rows):', training.size().getInfo())
print('Bandas usadas:', composite.bandNames().getInfo())

Training samples (rows): 159
Bandas usadas: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']


In [19]:
# Paso 4) entrenar clasificador Random Forest (100 árboles)

classifier = ee.Classifier.smileRandomForest(100).train(
    features=training,
    classProperty='landcover',
    inputProperties=composite.bandNames()
)

In [20]:
# Paso 5) Clasificar la imagen y mostrar clasificación

classified = composite.classify(classifier)

# Visualización
classVis = {'min': 0, 'max': 4, 'palette': ['blue', 'gray', 'green', 'violet', 'orange']}
Map2 = geemap.Map()
Map2.centerObject(roi, 10)
Map2.addLayer(classified.clip(roi), classVis, 'Imagen Clasificada')
Map2

Map(center=[-32.97411051812981, -60.747791764102], controls=(WidgetControl(options=['position', 'transparent_b…

In [21]:
# Paso 6) Accurracy assessment (matriz de confusión, OA, PA/UA, Kappa, F1)

# Sampleo sobre puntos de validación
test = classified.sampleRegions(
    collection=validationGCP,
    properties=['landcover'],
    scale=10
)

cm = test.errorMatrix('landcover', 'classification')

print('Confusion Matrix:\n', cm.getInfo())
print('Test Accuracy (Overall):', cm.accuracy().getInfo())
print('Producers Accuracy:', cm.producersAccuracy().getInfo())
print('Consumers Accuracy:', cm.consumersAccuracy().getInfo())

# Kappa correcto según API:
print('Kappa (API):', cm.kappa().getInfo())

# (Opcional) tu fórmula del guion original:
observedAccuracy = cm.accuracy()
expectedAccuracy = cm.kappa()  # en tu script lo llamabas "expected", aunque es kappa
kappa_formula = observedAccuracy.subtract(expectedAccuracy).divide(ee.Number(1).subtract(expectedAccuracy))
print('Kappa (según fórmula del script):', kappa_formula.getInfo())

# F1 para la clase 1 (urbano), como en tu script
print('F1 (clase 1):', cm.fscore(1).getInfo())

Confusion Matrix:
 [[30, 0, 0, 0, 0], [0, 32, 0, 0, 1], [0, 0, 9, 1, 0], [0, 0, 1, 13, 0], [0, 2, 0, 0, 13]]
Test Accuracy (Overall): 0.9509803921568627
Producers Accuracy: [[1], [0.9696969696969697], [0.9], [0.9285714285714286], [0.8666666666666667]]
Consumers Accuracy: [[1, 0.9411764705882353, 0.9, 0.9285714285714286, 0.9285714285714286]]
Kappa (API): 0.9352463179278822
Kappa (según fórmula del script): 0.2429834678969622
F1 (clase 1): [1, 0.955223880597015, 0.9, 0.9285714285714286, 0.896551724137931]
